In [1]:
from glio.loaders import nifti
from glio.jupyter_tools import show_slices, show_slices_arr
from glio.torch_tools import to_binary, center_of_mass, binary_erode3d
import torch
import polars as pl

In [2]:
from glio.python_tools import get_all_files
from glio.torch_tools import area_around
from glio.transforms import z_normalize
import os, h5py, numpy as np
cases = {}

for folder in sorted(os.listdir(r"E:\dataset\BRaTS2024-GoAT\ISBI2024-BraTS-GoAT-TrainingData")):
        print(folder, end='\r')
        full_path = os.path.join(r"E:\dataset\BRaTS2024-GoAT\ISBI2024-BraTS-GoAT-TrainingData", folder)
        filename = sorted(os.listdir(full_path))[0]
        case = filename[:len("BraTS-GoAT-00000")]
        cases[case] = {"id":case}
        for file in get_all_files(full_path):
            filename = os.path.basename(file)
            modality = filename[len("BraTS-GoAT-00000-"):].replace(".nii.gz", "")
            cases[case][modality] = file


        seg = nifti.read(cases[case]["seg"])
        center = center_of_mass(binary_erode3d(to_binary(seg), 2)).tolist()
        cases[case]["center"] = str(center)[1:-1]

        images = torch.stack([(z_normalize(nifti.read(cases[case][modality]).to(torch.float32)) if modality != "seg" else nifti.read(cases[case][modality])) for modality in ["t1c", "t1n", "t2f", "t2w", "seg"]])
        os.mkdir(rf"E:\dataset\BRaTS2024 2D\{case}")
        for i, slice in enumerate(area_around(images, center, (128,128,128)).unbind(1)):
            print(i, end='\r')
            np.savez_compressed(rf"E:\dataset\BRaTS2024 2D\{case}\{case}_{i}.npz", t1c=slice[0], t1n=slice[1], t2f=slice[2], t2w=slice[3], seg=slice[4])


In [3]:
df_dict = {"id":[], "seg":[], "t1c":[], "t1n":[], "t2f":[], "t2w":[], "center":[], "postop":[]}
for case, d in cases.items():
    df_dict["id"].append(case)
    df_dict["seg"].append(d["seg"])
    df_dict["t1c"].append(d["t1c"])
    df_dict["t1n"].append(d["t1n"])
    df_dict["t2f"].append(d["t2f"])
    df_dict["t2w"].append(d["t2w"])
    df_dict["center"].append(d["center"])
    postop = d["t1c"].split("-")[-2]
    if postop == "000": postop=0
    else: postop = 1
    df_dict["postop"].append(postop)


In [4]:
df = pl.from_dict(df_dict)
df

id,seg,t1c,t1n,t2f,t2w,center,postop
str,str,str,str,str,str,str,i64
"""BraTS-GoAT-000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""24.43157196044…",1
"""BraTS-GoAT-000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""57.09244155883…",1
"""BraTS-GoAT-000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""55.42643356323…",1
"""BraTS-GoAT-000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""93.43658447265…",1
"""BraTS-GoAT-000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""100.4669189453…",1
…,…,…,…,…,…,…,…
"""BraTS-GoAT-022…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""110.6450576782…",1
"""BraTS-GoAT-022…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""60.56921386718…",1
"""BraTS-GoAT-022…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""54.77987289428…",1


In [5]:
df.write_csv("BRaTS2024-GoAT.csv")

In [6]:
from glio.datasets.BRaTS2023 import BRaTS2023_GBM

ds = BRaTS2023_GBM()
ds.data

id,seg,t1c,t1n,t2f,t2w,center,postop,src
str,str,str,str,str,str,list[i64],i64,str
"""BraTS-GLI-0000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[70, 83, 139]",0,"""center"""
"""BraTS-GLI-0000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[80, 95, 140]",0,"""center"""
"""BraTS-GLI-0000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[107, 116, 133]",0,"""center"""
"""BraTS-GLI-0000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[104, 149, 146]",0,"""center"""
"""BraTS-GLI-0000…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[74, 74, 128]",0,"""center"""
…,…,…,…,…,…,…,…,…
"""BraTS-GLI-0166…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[105, 107, 98]",0,"""center"""
"""BraTS-GLI-0166…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[110, 101, 143]",0,"""center"""
"""BraTS-GLI-0166…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","""E:\dataset\BRa…","[90, 160, 149]",0,"""center"""
